# ModOAP - Classification automatique d'images

Ce script permet de classer les images contenues dans un dossier sur un Google Drive :


1. **Importer un modèle entraîné** à la classification d'images depuis un dossier Google Drive
  
  Le modèle est obtenu grâce au script *ModOAP - Classification automatique d'images - Entraînement*.

2. **Classer les images contenues dans un dossier** Google Drive 

  Les images sont alors copiées dans un dossier de résultats puis dans un sous-dossier nommé en fonction de la classe associée.

Ce script implémente la plateforme TensorFlow : https://www.tensorflow.org/

**Ce script doit être lancé dans un environnement d'exécution GPU : Exécution -> Modifier le type d'exécution -> GPU**

In [ ]:
#@markdown ### Synchronisation d'un compte Google Drive et imports:
#@markdown Lancer cette cellule, puis cliquer sur le lien généré par Google pour connecter un compte Drive si demandé.
# Connecter un compte Drive
from google.colab import drive
drive.mount('/content/drive')

#Imports
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import glob
import shutil
import pickle 


In [ ]:
#@markdown ### Chargement du modèle:
#@markdown Entrer le chemin du répertoire contenant le modèle entraîné, puis lancer la cellule.
dossier_modele = ""#@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/classification/modeles/modele123/

try :
  loaded_model = tf.keras.models.load_model(dossier_modele)
  class_names = pickle.loads(open(os.path.join(dossier_modele,"labels.pickle"), "rb").read())
  img_height = 180
  img_width = 180
  print("Le modèle a été chargé correctement")
  print("Etiquettes du modèle : ",class_names )
except :
  print("Problème lors du chargement du modèle")




In [ ]:
#@markdown ### Définir le dossier contenant les images à classer 
#@markdown Entrer le chemin du répertoire contenant directement les images à classer :
dossier_apredire = ""#@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/datasets/images_a_classer/

#@markdown Formats supportés : JPG, PNG, BMP, GIF 

#@markdown ### Définir le répertoire de destination 
#@markdown Entrer le chemin d'un nouveau répertoire où classer le corpus :
dossier_resultats = ""#@param {type:"string"}
#@markdown Exemple de chemin:
#@markdown /content/drive/My Drive/datasets/resultats_class

#@markdown ### Définir un seuil de confiance (facultatif) 
#@markdown Entrer un seuil en dessous duquel une image n'est pas associée à une classe. Le seuil est compris entre 0 et 100 (pourcentage de confiance). 
#@markdown Fixer un seuil élevé permet de privilégier la précision aux dépens du rappel (améliore la pertinence des images classées en prenant le risque de ne pas toutes les classer).
#@markdown Laisser vide pour ne pas fixer de seuil :
seuil = "80"#@param {type:"string"}
#@markdown Exemple de seuil:
#@markdown 80
seuil = int(seuil)

if not os.path.exists(dossier_resultats) :
  os.makedirs(dossier_resultats)

for classe in class_names :
  if not os.path.exists(os.path.join(dossier_resultats, classe)) :
    os.makedirs(os.path.join(dossier_resultats, classe))
  
if not os.path.exists(os.path.join(dossier_resultats, "non_classees")) :
  os.makedirs(os.path.join(dossier_resultats, "non_classees"))

fichiers_apredire = glob.glob(os.path.join(dossier_apredire, "*.*"))
print("Classification des images du répertoire : ", dossier_apredire)
print("Nombre d'images à classer : ", len(fichiers_apredire))
print("Classes possibles : ", class_names)

results = []
if seuil : 
  
  for apredire_path in fichiers_apredire :
    print("-----------------------")
    print(apredire_path)
    img = keras.preprocessing.image.load_img(
        apredire_path, target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    #print("score : ", score)
    if float(100 * np.max(score)) > seuil :
      print("Classe associée : {} avec {:.2f} pourcents de confiance."
          .format(class_names[np.argmax(score)], 100 * np.max(score)))
      
      shutil.copy(apredire_path, os.path.join(dossier_resultats, class_names[np.argmax(score)]))
      results.append(class_names[np.argmax(score)])

    else : 
      print("score trop faible : ", str(100 * np.max(score)))
      shutil.copy(apredire_path, os.path.join(dossier_resultats, "non_classees"))
      results.append("non_classees")
      print("image non classée")

else : 
  for apredire_path in fichiers_apredire :
    print("-----------------------")
    print(apredire_path)
    img = keras.preprocessing.image.load_img(
        apredire_path, target_size=(img_height, img_width)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = loaded_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print("Classe associée : {} avec {:.2f} pourcents de confiance."
          .format(class_names[np.argmax(score)], 100 * np.max(score)))
    
    shutil.copy(apredire_path, os.path.join(dossier_resultats, class_names[np.argmax(score)]))
    results.append(class_names[np.argmax(score)])

print("-----------------------")
print("-----------------------")
print("-----------------------")

print("Les images ont été copiées dans les dossiers correspondant aux étiquettes dans :")
print(dossier_resultats)


In [ ]:
#@markdown ### Visualiser la répartition des classes associées

from collections import Counter
compte = Counter(results)
plt.bar(compte.keys(),compte.values())